### 🛠️ Configuração de device

In [1]:
import torch

n_devices = torch.cuda.device_count()

device_ = "cpu" if not n_devices else "cuda"

### 🗃️ Recuperando o log do notebook 01_EA_DF

In [2]:
import json
import os

In [3]:
path_video = "/home/elton/Projetos/Translater_Video/teste"
name_video = "1 - Preparations.mp4"

name_audio_out = "audio_completo.wav"
path_out = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"

path_log = os.path.join(path_out, 'log', 'log.json')

In [4]:
log = ''
with open(path_log, 'r', encoding='utf-8') as file:
    log = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/elton/Projetos/Translater_Video/teste/resultado_teste/log/log.json'

### 🔄 Tradução

In [5]:
ling_input = 'en'
ling_output = 'pt'

In [6]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

/home/elton/Projetos/Translater_Video/.venv_core/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-04 16:35:10.071512: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-04 16:35:10.162210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-04 16:35:11.888771: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [41]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [57]:
def traducao (texto, lang_origem, lang_destino):
    tokenizer.src_lang = lang_origem # Idioma de origem
    encoded_hi = tokenizer(texto, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id(lang_destino)) # Idioma Destino
    traducao = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    
    return traducao[0]

In [58]:
for index, i in enumerate(log):
    texto = i['text_origem']
    
    result = traducao(texto, ling_input, ling_output)
    
    log[index]['text_destino'] = result

In [60]:
log[0]

{'start': 6.92,
 'end': 29.87,
 'path_audio': '/home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/separados_01/0.wav',
 'text_origem': " Before we can begin, it is necessary that both Cinema 4D and Redshift are already installed. We will do this, of course, via the Maxon app, as I'm sure you know. You can find the Maxon app directly on the Maxon website. We go to Products, directly to Downloads, and then you can download the app right up here. For Windows, Mac or Linux, you can also download it here below. Then we open the app.",
 'text_destino': 'Antes que possamos começar, é necessário que tanto Cinema 4D e Redshift já estejam instalados. faremos isso, é claro, através da aplicação Maxon, como eu sei. Você pode encontrar a aplicação Maxon diretamente no site Maxon. Nós vamos para Produtos, diretamente para Downloads, e então você pode baixar a aplicação diretamente aqui. Para Windows, Mac ou Linux, você também pode baixar aqui abaixo.'}

In [62]:
with open (path_log, 'w', encoding='utf-8') as file:
    file.write(json.dumps(log, ensure_ascii=False, indent=2))

In [ ]:
from app.configs import torch_config, read_json, write_json
import torch  
    
def translat (text:str):
    from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
    import logging
    logger = logging.getLogger('M2M100ForConditionalGeneration') 
    logger = logging.getLogger('M2M100Tokenizer') 
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    
    ling_input = 'en'  # <<--- Para futuras atualizações
    ling_output = 'pt' # <<--- Para futuras atualizações
    
    tokenizer.src_lang = ling_input # Idioma de origem
    encoded_hi = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id(ling_output)) # Idioma Destino
    traducao = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    
    return traducao[0]
    
            
def main(path_log:str):
    log = read_json(path_log)
    
    for index, i in enumerate(log[1]):
        texto = i['text_origem']
        
        result = translat(texto)
        
        log[1][index]['text_destino'] = result
    
    write_json(path_log, log)